In [1]:
from llama import llama_api
#from gpt4 import gpt4o_api
from score import comparison

e:\an\envs\ece1786\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
input_file = "example_input.txt"
reference_file = "example_truth.txt"


In [ ]:
# my_prompt = "Read the following document and write a summary that includes the background of the case in one paragraph, the procedural history in two paragraphs, and the court's decision and reasoning in one paragraph."
# print(my_prompt)

In [ ]:
# Summarize the most essential parts of the case
prompt = "State the background of the case in one paragraph in the following document."
facts = llama_api(prompt=prompt, input_file=input_file, output_file="llama_agent_facts.txt")
print("Facts done.")

prompt = "Summarize the procedural history in two paragraphs in the following document."
history = llama_api(prompt=prompt, input_file=input_file, output_file="llama_agent_history.txt")
print("history done.")

prompt = "State the court's decision and reasoning in one paragraph in the following document."
reasoning = llama_api(prompt=prompt, input_file=input_file, output_file="llama_agent_reasoning.txt")
print("reasoning done.")

# Get the more concise conclusions which are put at the front of the summary.
case_summary = "\n\n".join([facts, history, reasoning])

prompt = "In one simple clause, write a concise headline that summarizes the court's decision in the following summary:"
decision = llama_api(prompt=prompt, text=case_summary, output_file="llama_agent_decision.txt")
print("decision done.")

prompt = "In one simple sentence, write a concise legal statement that summarizes the central question in the following case summary:"
issue = llama_api(prompt=prompt, text=case_summary, output_file="llama_agent_issue.txt")
print("issue done.")

# Organize to form the final output summary.
combined_summary = "\n\n".join([decision, issue, facts, history, reasoning])

prompt = "Combine the following pieces of text into a single cohesive passage. Ensure the flow is smooth and logical while keeping most of the original content unchanged. Edit only as needed for readability and natural transitions"
final_output = llama_api(prompt=prompt, text=combined_summary, output_file="llama_final_output.txt")
print("combine done.")

In [3]:
comparison(reference_file, "llama_final_output.txt", "final_comparison.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'llama_final_output.txt'

## Combine Prompt

In [6]:
system_prompt = (
        "You are an expert legal assistant specializing in summarizing court decisions. "
        "Your role is to provide concise, clear, and accurate summaries of legal cases. "
        "Follow these rules for every response:\n"
        "1. Use formal language appropriate for legal and professional contexts.\n"
        "2. Ensure the summary is logically structured with clear sections:\n"
        "   - Title\n"
        "   - Introduction\n"
        "   - Background\n"
        "   - Court Proceedings\n"
        "   - Legal Principle\n"
        "   - Conclusion\n"
        "3. Avoid unnecessary repetition and ensure smooth transitions between sections.\n"
        "4. Provide summaries that are understandable to non-experts while retaining legal accuracy."
    )

agents_prompts = {
    "agent_1": (
        "Summarize the background of the case in one concise paragraph. Include the complainant, the accused, the alleged offences, and references to applicable sections of the Criminal Code."
        "Focus on the complainant, the accused, and the core allegations."
    ),
    "agent_2": (
        "Summarize the entire court proceedings in one concise paragraph, including: "
        "- The trial process: Describe the evidence presented, the trial judge's decision, and their reasoning. "
        "- The appellate process: Highlight the appellate arguments, the appellate courts' reasoning, and their decisions. "
        "- The final decision: Summarize the final judgment and reasoning of the highest court involved in this case. "
        "Focus on the logical progression of the case and key reasoning at each stage."
    ),
    "agent_3": (
        "Summarize the most critical legal principle established in this case in one concise sentence. "
        "Focus on the general relevance of the evidence, its relationship to the legal elements of the offence, and ensure the principle can apply broadly in similar cases."
    ),
    "agent_4": (
        "Write a concluding paragraph starting with 'Writing for the'. "
        "Summarize the majority opinion, focusing on the reasoning and its implications for the case."
    ),
}


# save results
generated_paragraphs = {}

# generate texts
for agent_name, prompt in agents_prompts.items():
    output_file = f"{agent_name}_output.txt"
    print(f"Running {agent_name}...")
    content = llama_api(
        prompt=prompt,
        input_file=input_file,
        output_file=output_file
    )
    generated_paragraphs[agent_name] = content
    print(f"{agent_name} completed. Output saved to {output_file}")

agent_5_prompt = (
    "Combine the following text outputs from seven agents into a single cohesive and well-structured summary. "
    "Follow this structure and ensure each section adheres to the specified format: "
    "1. Title: Begin with the name of the court and provide a concise summary of the decision. For example, 'The Supreme Court sets aside...'. "
    "2. Introduction: Write one sentence starting with 'This appeal' that summarizes the central legal question. "
    "3. Background: Summarize the case facts, including the complainant, the accused, the alleged offences, and references to relevant legal provisions. "
    "4. Court Proceedings: Describe the entire court process in one paragraph, including the trial, appellate decisions, and final judgment, ensuring smooth progression. "
    "5. Legal Principle: Provide the most critical legal principle established by the court in one concise sentence. Ensure it is general and applicable to similar cases. "
    "6. Conclusion: Write a concluding paragraph starting with 'Writing for the', summarizing the court's reasoning and its broader implications for the law. "
    "- Remove all redundancy or repetition while preserving critical legal principles and facts. "
    "- Ensure smooth transitions between sections, with each part contributing to a clear and logical narrative."
    "- Part 5 should be absolutely one sentence, and not include the specific cases or facts."
    "- part 6 should not be changed too much."
)

# combine all paragraphs
all_paragraphs_combined = "\n\n".join(generated_paragraphs.values())

# final
final_output = llama_api(
    prompt=agent_5_prompt,
    text=all_paragraphs_combined,
    output_file="final_case_summary_integrated.txt"
)
print("Final integrated summary saved to final_case_summary_integrated.txt.")


Running agent_1...


KeyError: 'LA-cafe700b5f334e16a6a1d5d07d8b76f16e4b01be7fc640e2a9e1847bb5796840'

In [5]:
# Define the system prompt to set the tone and role for the model
system_prompt = (
    "You are an expert legal assistant specializing in summarizing court decisions. "
    "Your role is to provide concise, clear, and accurate summaries of legal cases. "
    "Follow these rules for every response:\n"
    "1. Use formal language appropriate for legal and professional contexts.\n"
    "2. Ensure the summary is logically structured with clear sections:\n"
    "   - Title\n"
    "   - Introduction\n"
    "   - Background\n"
    "   - Court Proceedings\n"
    "   - Legal Principle\n"
    "   - Conclusion\n"
    "3. Avoid unnecessary repetition and ensure smooth transitions between sections.\n"
    "4. Provide summaries that are understandable to non-experts while retaining legal accuracy."
)

# Define agent-specific prompts
agents_prompts = {
    "agent_1": (
        "Summarize the background of the case in one concise paragraph. "
        "Include the complainant, the accused, the alleged offences, and references to applicable sections of the Criminal Code. "
        "Focus on the complainant, the accused, and the core allegations."
    ),
    "agent_2": (
        "Summarize the entire court proceedings in one concise paragraph, including: "
        "- The trial process: Describe the evidence presented, the trial judge's decision, and their reasoning. "
        "- The appellate process: Highlight the appellate arguments, the appellate courts' reasoning, and their decisions. "
        "- The final decision: Summarize the final judgment and reasoning of the highest court involved in this case. "
        "Focus on the logical progression of the case and key reasoning at each stage."
    ),
    "agent_3": (
        "Summarize the most critical legal principle established in this case in one concise sentence. "
        "Focus on the general relevance of the evidence, its relationship to the legal elements of the offence, and ensure the principle can apply broadly in similar cases."
    ),
    "agent_4": (
        "Write a concluding paragraph starting with 'Writing for the'. "
        "Summarize the majority opinion, focusing on the reasoning and its implications for the case."
    ),
}

# Integration agent prompt
agent_5_prompt = (
    "Combine the following text outputs from four agents into a single cohesive and well-structured summary. "
    "Follow this structure and ensure each section adheres to the specified format: "
    "1. Title: Begin with the name of the court and provide a concise summary of the decision. For example, 'The Supreme Court sets aside...'. "
    "2. Introduction: Write one sentence starting with 'This appeal' that summarizes the central legal question. "
    "3. Background: Summarize the case facts, including the complainant, the accused, the alleged offences, and references to relevant legal provisions. "
    "4. Court Proceedings: Describe the entire court process in one paragraph, including the trial, appellate decisions, and final judgment, ensuring smooth progression. "
    "5. Legal Principle: Provide the most critical legal principle established by the court in one concise sentence. Ensure it is general and applicable to similar cases. "
    "6. Conclusion: Write a concluding paragraph starting with 'Writing for the', summarizing the court's reasoning and its broader implications for the law. "
    "- Remove all redundancy or repetition while preserving critical legal principles and facts. "
    "- Ensure smooth transitions between sections, with each part contributing to a clear and logical narrative."
    "- Remove all the subtitles, like 'conclusion'. "
)

# Generate text for each agent
generated_paragraphs = {}

for agent_name, prompt in agents_prompts.items():
    output_file = f"{agent_name}_output.txt"
    print(f"Running {agent_name}...")

    # Combine system prompt and agent prompt
    combined_prompt = f"{system_prompt}\n\n{prompt}"
    # Assume `input_file` contains case details
    content = llama_api(
        prompt=combined_prompt,
        input_file=input_file,
        output_file=output_file
    )
    generated_paragraphs[agent_name] = content
    print(f"{agent_name} completed. Output saved to {output_file}")

# Combine all agent outputs
all_paragraphs_combined = "\n\n".join(generated_paragraphs.values())

print("Running final integration agent...")

# Combine system prompt with integration task
combined_integration_prompt = f"{system_prompt}\n\n{agent_5_prompt}"
final_output = llama_api(
    prompt=combined_integration_prompt,
    text=all_paragraphs_combined,
    output_file="final_case_summary_integrated.txt"
)
print("Final integrated summary saved to final_case_summary_integrated.txt.")


Running agent_1...
agent_1 completed. Output saved to agent_1_output.txt
Running agent_2...
agent_2 completed. Output saved to agent_2_output.txt
Running agent_3...
agent_3 completed. Output saved to agent_3_output.txt
Running agent_4...
agent_4 completed. Output saved to agent_4_output.txt
Running final integration agent...
Final integrated summary saved to final_case_summary_integrated.txt.


In [4]:
comparison(reference_file, "final_case_summary_integrated.txt", "test_score.txt")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "e:\an\envs\ece1786\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Rick\AppData\Local\Temp\ipykernel_15896\1909177251.py", line 1, in <module>
    comparison(reference_file, "final_case_summary_integrated.txt", "test_score.txt")
  File "e:\mie\1786\project\prompt_experiments\score.py", line 27, in comparison
    bert_precision, bert_recall, bert_f1 = calculate_bertscore(reference_summary, generated_summary)
  File "e:\mie\1786\project\prompt_experiments\score.py", line 14, in calculate_bertscore
    P, R, F1 = score([generated], [reference], lang="en", verbose=True)
  File "e:\an\envs\ece1786\lib\site-packages\bert_score\score.py", line 101, in score
    model.to(device)
  File "e:\an\envs\ece1786\lib\site-packages\transformers\modeling_utils.py", line 3157, in to
    return super().to(*args, **kwargs)
  File "e:\an\envs\ece1786\lib\site-packages

In [ ]:
# with open("multi_agent_concat_results/llama_output_facts.txt", "r") as file:
#     facts = file.read()

# with open("multi_agent_concat_results/llama_output_history.txt", "r") as file:
#     history = file.read()

# with open("multi_agent_concat_results/llama_output_reasoning.txt", "r") as file:
#     reasoning = file.read()

In [ ]:
# # Get the more concise conclusions which are put at the front of the summary.
# case_summary = "\n\n".join([facts, history, reasoning])

# prompt = "In one simple clause, write a concise headline that summarizes the court's decision in the following summary:"
# decision = llama_api(prompt=prompt, text=case_summary, output_file="llama_agent_decision.txt")
# print("decision done.")

# prompt = "In one simple sentence, write a concise legal statement that summarizes the central question in the following case summary:"
# issue = llama_api(prompt=prompt, text=case_summary, output_file="llama_agent_issue.txt")
# print("issue done.")

# # Organize to form the final output summary.
# combined_summary = "\n\n".join([decision, issue, facts, history, reasoning])

# prompt = "Combine the following pieces of text into a single cohesive passage. Ensure the flow is smooth and logical while keeping most of the original content unchanged. Edit only as needed for readability and natural transitions"
# final_output = llama_api(prompt=prompt, text=combined_summary, output_file="llama_final_output.txt")
# print("combine done.")